In [40]:
%pip install mediapipe opencv-python pandas scikit-learn
%pip install --upgrade tensorflow tensorflow-addons


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow info messages

In [42]:
import mediapipe as mp 
import cv2 

In [44]:
mp_drawing = mp.solutions.drawing_utils # provides utility functions for rendering landmarks and connections on images.
mp_holistic = mp.solutions.holistic #provides a holistic (full-body) pose estimation solution using a combination of face, hand, and pose landmarks.

In [45]:
# Open the camera (you can change the index as needed)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Break the loop if there's an issue with the camera
        if not ret:
            print("Error: Could not read frame.")
            break

        # Recolor Feed: Captures a frame from the video feed and converts it to RGB color space.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Processes the RGB frame using the holistic model to obtain pose and hand landmarks.
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks on the image
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Holistic Model', image)

        # Exit the program when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1700497308.978400       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro


In [46]:
import csv
import numpy as np
import os

In [47]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)

In [48]:
#  calculated as the sum of the number of landmarks in the pose and face detection results. Each landmark is represented by a set of coordinates (x, y, z) in 3D space.

In [49]:
num_coords

501

In [50]:
landmarks = ['class']
for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val) ,'y{}'.format(val),'z{}'.format(val),'v{}'.format(val) ]

<!-- list named landmarks is initialized with the first element being the string 'class'. This suggests that the first column in the CSV file will represent the class or label associated with each set of landmarks.

The code then enters a loop that iterates over the range from 1 to num_coords + 1. Inside the loop, for each value val, four elements are appended to the landmarks list using string formatting. -->

In [54]:
with open('coords.csv' , mode='w',newline='') as f:
    csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [59]:
class_name = "Sad"

In [60]:
cap = cv2.VideoCapture(0)

# initialize the model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret , frame = cap.read()

        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        image.flags.writeable = False

        results = holistic.process(image)

        # print(results.pose_landmarks)
        image.flags.writeable = True

        image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)

        # face
        mp_drawing.draw_landmarks(image , results.face_landmarks , mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10) ,thickness = 1 ,circle_radius=1 ) ,         mp_drawing.DrawingSpec(color=(80,256,121) ,thickness = 1 ,circle_radius=1 ))

        # right hand
        mp_drawing.draw_landmarks(image , results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10) ,thickness = 2 ,circle_radius=4 ) ,          mp_drawing.DrawingSpec(color=(80,44,121) ,thickness = 2 ,circle_radius=2 ) )
        
        # left hand
        mp_drawing.draw_landmarks(image , results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76) ,thickness = 2 ,circle_radius=4 ) ,         mp_drawing.DrawingSpec(color=(121,44,230) ,thickness = 2 ,circle_radius=2 ))
        
        # pose detection
        mp_drawing.draw_landmarks(image , results.pose_landmarks , mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66) ,thickness = 2 ,circle_radius=4 ) ,        mp_drawing.DrawingSpec(color=(245,66,230) ,thickness = 2 ,circle_radius=2 ))

        # export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x , landmark.y , landmark.z , landmark.visibility] for landmark in face]).flatten())

            row = pose_row + face_row
            row.insert(0 , class_name)

            with open('coords.csv' , mode='a',newline='') as f:
                csv_writer = csv.writer(f , delimiter=',',quotechar='"',quoting = csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        cv2.imshow("Holistic Model Detections",image)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1700497404.859961       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro


In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_csv('coords.csv')

In [63]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Surprised,0.488796,0.556369,-0.690917,0.999989,0.509879,0.490774,-0.666102,0.999952,0.520998,...,-0.006797,0.0,0.543818,0.487278,0.009836,0.0,0.547826,0.479473,0.010367,0.0
1,Surprised,0.492625,0.561033,-0.624011,0.999990,0.512042,0.493424,-0.597082,0.999956,0.522981,...,-0.005952,0.0,0.544458,0.487618,0.010647,0.0,0.548437,0.480052,0.011229,0.0
2,Surprised,0.493200,0.561374,-0.549691,0.999991,0.512050,0.493667,-0.519480,0.999960,0.522974,...,-0.006757,0.0,0.542524,0.488541,0.009856,0.0,0.546430,0.481206,0.010410,0.0
3,Surprised,0.493869,0.562714,-0.597898,0.999992,0.512039,0.494485,-0.571086,0.999964,0.522964,...,-0.006860,0.0,0.542472,0.488226,0.010139,0.0,0.546405,0.481036,0.010699,0.0
4,Surprised,0.494129,0.567248,-0.582420,0.999992,0.512077,0.498165,-0.553904,0.999967,0.522976,...,-0.006555,0.0,0.542720,0.488726,0.010197,0.0,0.546644,0.481283,0.010730,0.0


In [64]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
195,Sad,0.477976,0.685334,-1.668706,0.999352,0.497595,0.621794,-1.637679,0.999038,0.508880,...,-0.008341,0.0,0.524279,0.608423,0.010936,0.0,0.528584,0.603083,0.011373,0.0
196,Sad,0.477780,0.685252,-1.640848,0.999388,0.497315,0.621637,-1.602010,0.999083,0.508657,...,-0.008596,0.0,0.520966,0.614864,0.010859,0.0,0.525369,0.608767,0.011459,0.0
197,Sad,0.472393,0.684212,-1.440776,0.999411,0.492305,0.621525,-1.421002,0.999112,0.503412,...,-0.009204,0.0,0.518874,0.617339,0.009622,0.0,0.523429,0.610770,0.010141,0.0
198,Sad,0.470516,0.685309,-1.437572,0.999441,0.489938,0.622213,-1.418694,0.999149,0.500857,...,-0.008493,0.0,0.518234,0.618711,0.010537,0.0,0.522774,0.612458,0.011028,0.0
199,Sad,0.468265,0.685298,-1.457847,0.999475,0.487545,0.622235,-1.442625,0.999194,0.498440,...,-0.009043,0.0,0.516515,0.617896,0.010381,0.0,0.520977,0.611535,0.010871,0.0


In [66]:
x = df.iloc[: , 1:]
y = df.iloc[: , 0]

In [67]:
X_train , X_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state=0)

In [68]:
len(X_train) , len(X_test)

(160, 40)

In [69]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [70]:
pipelines = {
    'lr':make_pipeline(StandardScaler() , LogisticRegression()),
    'rc':make_pipeline(StandardScaler() , RidgeClassifier()),
    'rf':make_pipeline(StandardScaler() , RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler() , GradientBoostingClassifier())
}

In [71]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [72]:
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model = pipeline.fit(X_train, y_train)

In [73]:
fit_models = {}
for algo, pipeline in pipelines.items():
    fit_models[algo] = model

In [74]:
fit_models['rc'].predict(X_test)

array(['Surprised', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Surprised',
       'Sad', 'Surprised', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Surprised', 'Surprised', 'Sad', 'Surprised', 'Happy', 'Sad',
       'Sad', 'Surprised', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Surprised', 'Surprised', 'Surprised', 'Sad', 'Sad', 'Surprised',
       'Sad', 'Sad', 'Surprised', 'Happy', 'Sad'], dtype=object)

In [75]:
fit_models['lr'].predict(X_test)

array(['Surprised', 'Sad', 'Sad', 'Happy', 'Sad', 'Sad', 'Surprised',
       'Sad', 'Surprised', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Surprised', 'Surprised', 'Sad', 'Surprised', 'Happy', 'Sad',
       'Sad', 'Surprised', 'Sad', 'Happy', 'Sad', 'Sad', 'Sad', 'Sad',
       'Surprised', 'Surprised', 'Surprised', 'Sad', 'Sad', 'Surprised',
       'Sad', 'Sad', 'Surprised', 'Happy', 'Sad'], dtype=object)

In [76]:
from sklearn.metrics import accuracy_score
import pickle

In [77]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [78]:
y_test

18     Surprised
170          Sad
107          Sad
98         Happy
177          Sad
182          Sad
5      Surprised
146          Sad
12     Surprised
152          Sad
61         Happy
125          Sad
180          Sad
154          Sad
80         Happy
7      Surprised
33     Surprised
130          Sad
37     Surprised
74         Happy
183          Sad
145          Sad
45     Surprised
159          Sad
60         Happy
123          Sad
179          Sad
185          Sad
122          Sad
44     Surprised
16     Surprised
55     Surprised
150          Sad
111          Sad
22     Surprised
189          Sad
129          Sad
4      Surprised
83         Happy
106          Sad
Name: class, dtype: object

In [79]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)
# save the trained logistic regression model 

In [80]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [81]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [82]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates:Extracts pose and face landmarks, flattens them, and concatenates them into a row.
        # Creates a DataFrame (X) with the row.
        # Uses the trained model (fit_models['lr']) to predict the body language class and probability.
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            #Extracts the coordinates of the left ear landmark detected by the Holistic model.
            # The coordinates are scaled based on the resolution (640x480) of the video frame.
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1700497435.764353       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1 Pro
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.67089285 0.20158346 0.1275237 ]
Happy [0.54677509 0.28786556 0.16535935]
Happy [0.55258897 0.27795561 0.16945542]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

Happy [0.46247865 0.35545003 0.18207132]
Sad [0.38027077 0.48414967 0.13557956]
Happy [0.49549829 0.33683423 0.16766748]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

Happy [0.59977897 0.00367531 0.39654572]
Happy [0.77443041 0.00451933 0.22105026]
Happy [0.54718944 0.00692969 0.44588087]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have va

Surprised [0.34443045 0.02219042 0.63337913]
Happy [0.54728053 0.01587919 0.43684028]
Surprised [0.10055288 0.03991666 0.85953047]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [83]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(367, 261)